In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import silhouette_samples, silhouette_score, v_measure_score
from sklearn.datasets import load_files
import re
from unidecode import unidecode
from mpl_toolkits import mplot3d
from yellowbrick.cluster import KElbowVisualizer
print("Importado1")


nltk.download('rslp')
nltk.download('stopwords')


Importado1


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:

# Carregue os DataFrames origem_ID e text do CSV


caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )


In [9]:
# Função para encontrar a correspondência fuzzy e retornar a correspondência com o maior score
def find_best_match(row):
    matches = process.extractOne(row['classif'], text['dsc_item'], scorer=fuzz.ratio)
    return pd.Series([row['classif'], matches[0]])

# Aplicar a função de correspondência fuzzy e criar o novo DataFrame
matches_df = origem_ID.apply(find_best_match, axis=1)
matches_df.columns = ['classif', 'melhor_classif']

# Salvar o DataFrame resultante em um novo CSV
matches_df.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(matches_df.head())




KeyboardInterrupt: 

In [4]:
#Limpando os textos e comparando

text['dsc_item'] = text['dsc_item'].astype(str)
origem_ID['classif'] = origem_ID['classif'].astype(str)
text["dsc_item"] = text["dsc_item"].apply(lambda x: x.lower()) #converte todas as letras para minúsculo
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: x.lower()) #converte todas as letras para minúsculo 
text["dsc_item"] = text["dsc_item"].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
text["dsc_item"] = text["dsc_item"].apply(lambda x: unidecode(x)) #remove acentos
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: unidecode(x)) #remove acentos
stemmer = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
text["dsc_item"] = text["dsc_item"].apply(lambda x: stemmer.stem(x))
stemmer1 = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: stemmer1.stem(x))

# Função para encontrar a correspondência fuzzy e retornar a correspondência com o maior score
def find_best_match(row):
    matches = process.extractOne(row['classif'], text['dsc_item'], scorer=fuzz.ratio)
    return pd.Series([row['classif'], matches[0], matches[1]])

# Aplicar a função de correspondência fuzzy e criar um novo DataFrame com os resultados
resultados = origem_ID.apply(find_best_match, axis=1)
resultados.columns = ['classif_origem', 'melhor_dst_item', 'score']

# Filtre apenas as linhas com um score acima de um determinado limite (opcional)
score_limite = 80  # Você pode ajustar esse limite conforme necessário
resultados_filtrados = resultados[resultados['score'] >= score_limite]

# Salvar o DataFrame resultante em um novo CSV
resultados_filtrados.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(resultados_filtrados.head())


           classif_origem          melhor_dst_item  score
6519   reparo bomba d'agu  reparo da bomba d'agua*     90
6811   reparo bomba d'agu  reparo da bomba d'agua*     90
6929   reparo bomba d'agu  reparo da bomba d'agua*     90
7061   reparo bomba d'agu  reparo da bomba d'agua*     90
7194   reparo bomba d'agu  reparo da bomba d'agua*     90
